In [19]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch

In [14]:
programas = pd.read_json("../scraper_siglas-uc/outputs/programas_clean.json", orient="table")
corpus = programas["programa"].to_list()

In [23]:
programas.head()

,escuela,sigla,programa
0,Actuación,ACO4001,curso propone acercamiento concepto performanc...
1,Actuación,ACO256E,curso taller destinado adquirir tecnica improv...
2,Estudios Urbanos,IEU2002,curso compromete integridad academica reconoci...
3,Matemáticas,EPG4020,monitoring activity minimo candidato debe prep...
4,Matemáticas,MAT1023,curso desarrolla elementos basicos calculo enf...


In [16]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
documment_embs = sbert_model.encode(corpus, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 32/32 [01:56<00:00,  3.63s/it]


In [21]:
queries = ["machine learning with python"]

In [28]:
query_embedding = sbert_model.encode(queries[0], convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, documment_embs)[0]
top_results = torch.topk(cos_scores, k=10)

In [30]:
cos_scores.shape

torch.Size([1024])

In [27]:


print("\n\n======================\n\n")
print("Query:", queries[0])
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    print(programas.iloc[idx.numpy(), 1], "(Score: {:.4f})".format(score))

"""
# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
"""





Query: machine learning with python

Top 5 most similar sentences in corpus:


ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [21]:
sim_mtx = cosine_similarity(documment_embs)
sim_mtx = pd.DataFrame(sim_mtx, columns=programas["sigla"].values).set_index(programas["sigla"])

In [ ]:
sim_mtx.to_json("files/bert_simmtx.json")

In [30]:
sim_mtx["MAT1610"].sort_values(ascending=False).head(10)

sigla
MAT1610    1.000000
ICM2028    0.929249
AQC0300    0.928825
ART0403    0.925336
ARQ3626    0.923576
ICP0201    0.923245
FIZ0322    0.920672
EDU370F    0.919696
AQT0300    0.917334
ICE2313    0.915324
Name: MAT1610, dtype: float32